In [41]:
from statsmodels.tsa.api import AutoReg, VAR
from stargazer.stargazer import Stargazer
from geopy.distance import great_circle
import matplotlib.pyplot as plt
import sklearn.metrics as skm
from tqdm import tqdm
import pandas as pd
import numpy as np

In [42]:
data = pd.read_csv(r"/Users/main/Vault/Thesis/Data/pm25_weer.csv")
data.drop(data.iloc[:, 0:7], axis=1, inplace=True)
data.drop(["jaar", "maand", "weeknummer", "#STN", "timestamp", "components", "dag", "tijd", "T", "datum", "H", "T", "weekdag", "FH", "sensortype"], axis=1, inplace=True)
# data["Wind"] = np.cos(data["DD"]) * (data["U"] * 1.852)

In [54]:
grouped_df = data.groupby(["YYYYMMDD", "tag"])["pm25", "longitude", "latitude", "U", "DD"].mean().copy().reset_index()
grouped_df.rename(columns={"U":"Wind", "DD":"Angle"}, inplace=True)
grouped_df.head(5)

/var/folders/7f/x1mp2wjd10vcy_krqb5_kwjc0000gn/T/ipykernel_36389/2991179277.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_df = data.groupby(["YYYYMMDD", "tag"])["pm25", "longitude", "latitude", "U", "DD"].mean().copy().reset_index()


,YYYYMMDD,tag,pm25,longitude,latitude,Wind,Angle
0,20190625,Amsterdam,16.602565,4.866208,52.359714,69.739130,170.434783
1,20190625,Beverwijk,18.822000,4.650824,52.492369,77.400000,276.000000
2,20190626,Amsterdam,21.974609,4.866208,52.359714,81.043478,126.521739
3,20190626,Beverwijk,2.851417,4.657933,52.485871,77.333333,19.166667
4,20190627,Amsterdam,4.484500,4.866208,52.359714,73.458333,37.916667


In [44]:
Locations = grouped_df["tag"].unique()
LocDict = dict()

for i in range(len(Locations)):
    LocDict[Locations[i]] = (grouped_df[grouped_df.tag == Locations[i]]["latitude"].mean(), grouped_df[grouped_df.tag == Locations[i]]["longitude"].mean())

LocDict

{'Amsterdam': (52.385565471780744, 4.901327224364789),
 'Beverwijk': (52.486366756022534, 4.658584737682753),
 'Heemskerk': (52.50593616684639, 4.671953092146708),
 'Wijk aan Zee': (52.493443775747366, 4.59824578234632),
 'Velsen-Noord': (52.47382287197285, 4.647238399148998),
 'Driehuis': (52.447194827537714, 4.636977030738782),
 'IJmuiden': (52.45820717461739, 4.615567828011168),
 'Velsen-Zuid': (52.46234499999992, 4.637255999999981),
 'Zaandam': (52.45848175659801, 4.824912486862412),
 'Koog aan de Zaan': (52.46407900000035, 4.811263999999972),
 'Uithoorn': (52.23809099999961, 4.8082289999999865)}

In [66]:
def angle_between(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))

W = np.zeros((11, 11))
AngleMatrix = np.zeros((11, 11))

for i in range(len(LocDict)):
    for j in range(len(LocDict)):
        if i != j:
            theta = angle_between(LocDict[Locations[i]], LocDict[Locations[j]])
            W[i, j] = 1 / great_circle(LocDict[Locations[i]], LocDict[Locations[j]]).km
            AngleMatrix[i, j] = theta
            
pd.DataFrame(AngleMatrix)

,0,1,2,3,4,5,6,7,8,9,10
0,0.000000,0.273012,0.260418,0.339043,0.284098,0.292664,0.316923,0.293814,0.090137,0.105475,0.086235
1,359.726988,0.000000,359.987407,0.066031,0.011086,0.019653,0.043911,0.020802,359.817125,359.832463,359.813224
2,359.739582,0.012593,0.000000,0.078624,0.023679,0.032246,0.056504,0.033395,359.829718,359.845056,359.825817
3,359.660957,359.933969,359.921376,0.000000,359.945055,359.953622,359.977880,359.954771,359.751094,359.766432,359.747193
4,359.715902,359.988914,359.976321,0.054945,0.000000,0.008567,0.032825,0.009716,359.806039,359.821377,359.802138
5,359.707336,359.980347,359.967754,0.046378,359.991433,0.000000,0.024258,0.001149,359.797472,359.812811,359.793571
6,359.683077,359.956089,359.943496,0.022120,359.967175,359.975742,0.000000,359.976891,359.773214,359.788552,359.769313
7,359.706186,359.979198,359.966605,0.045229,359.990284,359.998851,0.023109,0.000000,359.796323,359.811661,359.792422
8,359.909863,0.182875,0.170282,0.248906,0.193961,0.202528,0.226786,0.203677,0.000000,0.015338,359.996099
9,359.894525,0.167537,0.154944,0.233568,0.178623,0.187189,0.211448,0.188339,359.984662,0.000000,359.980760


In [56]:
grouped_df["Date"] = grouped_df["YYYYMMDD"].astype(str)
grouped_df.set_index("Date", inplace=True)
grouped_df.drop(columns=["YYYYMMDD", "latitude", "longitude", "Angle"], inplace=True)
grouped_df.head(5)

,tag,pm25,Wind,Angle
Date,,,,
20190625,Amsterdam,16.602565,69.739130,170.434783
20190625,Beverwijk,18.822000,77.400000,276.000000
20190626,Amsterdam,21.974609,81.043478,126.521739
20190626,Beverwijk,2.851417,77.333333,19.166667
20190627,Amsterdam,4.484500,73.458333,37.916667


In [61]:
UniqueNames = grouped_df.tag.unique()

PolDict = {elem : pd.DataFrame() for elem in UniqueNames}
WindDict = {elem : pd.DataFrame() for elem in UniqueNames}
AngleDict = {elem : pd.DataFrame() for elem in UniqueNames}

for key in PolDict.keys():
    PolDict[key] = grouped_df[:][grouped_df.tag == key]
    WindDict[key] = grouped_df[:][grouped_df.tag == key]
    AngleDict[key] = grouped_df[:][grouped_df.tag == key]
    WindDict[key].drop(["pm25", "Angle"], axis=1, inplace=True)
    PolDict[key].drop(["Angle", "Wind"], axis=1, inplace=True)
    AngleDict[key].drop(["pm25", "Wind"], axis=1 , inplace=True)

for key in WindDict:
    PolDict[key].rename(columns={"pm25":key}, inplace=True)
    WindDict[key].rename(columns={"Wind":key}, inplace=True)
    AngleDict[key].rename(columns={"Angle":key}, inplace=True)
    del PolDict[key]["tag"]
    del WindDict[key]["tag"]
    del AngleDict[key]["tag"]


type(AngleDict["Amsterdam"])

pandas.core.frame.DataFrame

In [63]:
df_pol = pd.DataFrame(PolDict["Amsterdam"].copy())
df_wind = pd.DataFrame(WindDict["Amsterdam"].copy())
df_angle = pd.DataFrame(AngleDict["Amsterdam"].copy())

for key in PolDict:
    df_pol = df_pol.combine_first(PolDict[key])
    df_wind = df_wind.combine_first(WindDict[key])
    df_angle = df_angle.combine_first(AngleDict[key])

df_angle

,Amsterdam,Beverwijk,Driehuis,Heemskerk,IJmuiden,Koog aan de Zaan,Uithoorn,Velsen-Noord,Velsen-Zuid,Wijk aan Zee,Zaandam
Date,,,,,,,,,,,
20190625,170.434783,276.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190626,126.521739,19.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190627,37.916667,33.013699,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190628,49.583333,50.314961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20190629,117.083333,118.232044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20221018,176.828194,180.424028,177.058824,173.281250,168.938053,NaN,177.058824,177.058824,NaN,177.058824,177.058824
20221019,76.592357,77.437186,77.083333,77.111111,77.083333,NaN,77.083333,77.083333,NaN,77.083333,77.083333
20221020,128.169014,127.835616,127.727273,127.245509,127.777778,NaN,127.727273,127.727273,NaN,127.727273,127.727273


In [64]:
for column in df_pol:
    median_value = (df_pol[column].median(), df_wind[column].median(), df_angle[column].median())
    df_pol[column].fillna(value=median_value[0], inplace = True)
    df_wind[column].fillna(value=median_value[1], inplace = True)
    df_angle[column].fillna(value=median_value[2], inplace = True)
    
df_wind

,Amsterdam,Beverwijk,Driehuis,Heemskerk,IJmuiden,Koog aan de Zaan,Uithoorn,Velsen-Noord,Velsen-Zuid,Wijk aan Zee,Zaandam
Date,,,,,,,,,,,
20190625,69.739130,77.400000,78.852273,79.087560,79.125000,78.1,76.875000,78.708333,78.666667,78.952941,78.915254
20190626,81.043478,77.333333,78.852273,79.087560,79.125000,78.1,76.875000,78.708333,78.666667,78.952941,78.915254
20190627,73.458333,73.150685,78.852273,79.087560,79.125000,78.1,76.875000,78.708333,78.666667,78.952941,78.915254
20190628,71.833333,71.259843,78.852273,79.087560,79.125000,78.1,76.875000,78.708333,78.666667,78.952941,78.915254
20190629,58.166667,57.254144,78.852273,79.087560,79.125000,78.1,76.875000,78.708333,78.666667,78.952941,78.915254
...,...,...,...,...,...,...,...,...,...,...,...
20221018,83.555066,83.127208,83.411765,83.710938,83.831858,78.1,83.411765,83.411765,78.666667,83.411765,83.411765
20221019,82.245223,81.804020,82.083333,82.138889,82.083333,78.1,82.083333,82.083333,78.666667,82.083333,82.083333
20221020,93.056338,92.852055,92.863636,92.754491,92.888889,78.1,92.863636,92.863636,78.666667,92.863636,92.863636


In [ ]:
VARModel = VAR(df_pol).fit()
VARModel.summary()

In [ ]:
WY = pd.DataFrame(np.matmul(df_pol.to_numpy(), W))

i = 0
for key in PolDict:
    WY.rename(columns={i:f'{key}'}, inplace=True)
    i += 1

SVAR = VAR(WY).fit()
SVAR.summary()

In [16]:
WW = list()

for i in range(len(df_wind)):
    wind = np.zeros((len(df_wind.columns), len(df_wind.columns)))
    
    for j in range(len(df_wind.columns)):

        if df_wind.iloc[i, j] != 0:
            wind[:, j] = W[:, j] + 1 / df_wind.iloc[i, j]
        else:
            wind[:, j] = W[:, j]

    WW.append(wind)

WWY = np.zeros((len(df_pol), len(df_pol.columns)))

for i in range(len(df_pol)):
    WWY[i, :] = np.matmul(df_pol.iloc[i, :].to_numpy(), WW[i])

WWY = pd.DataFrame(WWY)

SWVAR = VAR(WWY).fit()
SWVAR.summary()

In [19]:
for key in PolDict:
    R2 = skm.r2_score(SWVAR.fittedvalues[key] + SWVAR.resid[key], SWVAR.fittedvalues[key])
    print(F'The R-Squared of {key} is: {R2*100:.2f}%')

    # y = SWVAR.fittedvalues['foobar'] + SWVAR.resid['foobar']
    # R2 = 1 - np.sum(SWVAR.resid['foobar'].values ** 2) / np.sum((y.values - y.mean()) ** 2)

KeyError: 'Amsterdam'